In [934]:
import pandas as pd
import numpy as np
import json
import transformers

import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# http://insideairbnb.com/get-the-data/

In [839]:
# Import datasets 

listing = pd.read_csv('http://data.insideairbnb.com/china/hk/hong-kong/2022-06-14/data/listings.csv.gz')
review = pd.read_csv('http://data.insideairbnb.com/china/hk/hong-kong/2022-06-14/data/reviews.csv.gz')
vacancy = pd.read_csv('http://data.insideairbnb.com/china/hk/hong-kong/2022-06-14/data/calendar.csv.gz')

listingViz = pd.read_csv('http://data.insideairbnb.com/china/hk/hong-kong/2022-06-14/visualisations/listings.csv')
reviewDates = pd.read_csv('http://data.insideairbnb.com/china/hk/hong-kong/2022-06-14/visualisations/reviews.csv')

In [310]:
listingViz.neighbourhood.value_counts()

Yau Tsim Mong        2079
North                1460
Wan Chai             1336
Central & Western     992
Yuen Long             536
Islands               308
Kowloon City          243
Eastern               202
Sham Shui Po          146
Sai Kung               96
Southern               49
Sha Tin                47
Tuen Mun               39
Kwun Tong              31
Tai Po                 27
Tsuen Wan              17
Kwai Tsing              7
Wong Tai Sin            5
Name: neighbourhood, dtype: int64

In [411]:
item = listingViz[listingViz.id == 51677836].index[0]
address, (latitude, longitude) = geolocator.reverse((listingViz.latitude[item], listingViz.longitude[item]), language='en')
address.split(', ')[-2]

'518000'

### Data Preproc: Removing non-HK listings

In [300]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
address, (latitude, longitude) = geolocator.reverse("22.24072, 113.97625", language='en')
address.split(', ')

In [452]:
# districts that may contain non-HK listings, ie Shenzhen addresses

dfSearch = listingViz.loc[listingViz.neighbourhood.isin(['North', 'Yuen Long', 'Tsuen Wan', 'Tai Po', 'Tuen Mun']), : ]

In [453]:
dfSearch.shape

(2079, 18)

In [454]:
# listings are Chinese addresses

chinaId = list()

for i in range(len(dfSearch)):
    address, (latitude, longitude) = geolocator.reverse((dfSearch.latitude.iloc[i], dfSearch.longitude.iloc[i]), language='en')
    if address.split(', ')[-2] != 'Hong Kong':
        chinaId.append(dfSearch.id.iloc[i])
        
len(chinaId)

1947

In [456]:
# how many reviews out of >100K are non-HK
counter = 0 

for i in review.listing_id:
    if i in chinaId:
        counter+=1
counter

21998

In [480]:
len(review) #97158 expected

119156

In [487]:
reviewHk = review.copy()

In [489]:
for i in chinaId:
    reviewHk.drop(reviewHk[reviewHk.listing_id == i].index, inplace=True)

#reviewHK.to_csv('./data/reviewHk.csv', index=False)
reviewHk.shape

(97158, 6)

In [502]:
listingHk = listing.copy()

for i in chinaId:
    listingHk.drop(listingHk[listingHk.id == i].index, inplace=True)

listingHk.to_csv('./data/listingHk.csv', index=False)
listingHk.shape

(5673, 74)

In [501]:
vacancyHk = vacancy.copy()

for i in chinaId:
    vacancyHk.drop(vacancyHk[vacancyHk.listing_id == i].index, inplace=True)

vacancyHk.to_csv('./data/vacancyHk.csv', index=False)
vacancyHk.shape

(2070280, 7)

In [503]:
listingVizhk = listingViz.copy()

for i in chinaId:
    listingVizhk.drop(listingVizhk[listingVizhk.id == i].index, inplace=True)

listingVizhk.to_csv('./data/listingVizhk.csv', index=False)
listingVizhk.shape

(5673, 18)

### Experiment with Summarization Pipeline

In [877]:
from transformers import pipeline

summarizer = pipeline('summarization', model='knkarthick/MEETING_SUMMARY', max_length=48)

In [854]:
reviewHkEn = pd.read_csv('./data/reviewHkEn.csv')

In [940]:
text = reviewHkEn.commentsEn[95768]
#text = "This studio off-apart are approximately 210 sqft, have double bed, 1 bathroom with separated shower, located in 4th floor with building lift, security key card access to main door and to your room, with natural light, work desk spacious and storage. Located in heart of Tsim Sha Tsui, a district decorated with Hong Kong's famous neon lights. Around the corner from the MTR and multiple bus links, supermarkets and shopping malls."
print(text, '\n\n', 'summarized text:\n',summarizer(text)[0].get('summary_text'))

mabel and ramon have an amazing villa! we spent two wonderful night with them and their lovely dog. you can enjoy romantic barbecue night and lovely breakfast along the poolside, they even prepared the candle!   a real nice escape from hustle and bustle, surrounded by nature (you will find squirrels and perfect sunshine while breakfast), definitely recommended! 

 summarized text:
 Mabel and Ramon have an amazing villa. They invited us for a romantic barbecue night and breakfast at the poolside.


### Experiment with Zero Shot Text Classifiers

In [12]:
# https://analyticsindiamag.com/a-complete-tutorial-on-zero-shot-text-classification/

In [4]:
classifier = transformers.pipeline('zero-shot-classification', 
                                   model='facebook/bart-large-mnli')

In [838]:
rev[0]

"the apartment on holly wood rd was exactly as described. it was a comfortable and very convenient base for our two week holiday. our host was very forthcoming in answering queries and giving us some local tips. i would definitely recommend candace's place to friends and would have no hesitation in booking it again when planning another trip to hk."

In [ ]:
# family vs romantic etc, near city, quiet, etc

In [687]:
sequence_to_classify = reviewHkEn.commentsEn[:100]
candidate_labels = ['central location', 'good comfort', 'strict', 'amenities', 'peaceful']
classifier(sequence_to_classify, candidate_labels, multi_label=False)

{'sequence': "the apartment on holly wood rd was exactly as described. it was a comfortable and very convenient base for our two week holiday. our host was very forthcoming in answering queries and giving us some local tips. i would definitely recommend candace's place to friends and would have no hesitation in booking it again when planning another trip to hk.",
 'labels': ['good comfort',
  'amenities',
  'central location',
  'peaceful',
  'strict'],
 'scores': [0.6978234052658081,
  0.1472613364458084,
  0.08935888111591339,
  0.050628989934921265,
  0.014927397482097149]}

In [198]:
classifier1(sequence_to_classify, candidate_labels, multi_label=False)   

{'sequence': 'ROOM for 2 persons !!!  right in Wanchai.<br />with LIFT.<br /><br />it is soooo close to Times Square. There is a nice cinema there so you can walk there in your pyjamas to go and catch a movie and just WALK HOME after !<br /><br />many FRESH meat and FRESH vegetables and fruits just 3 mins walk away.<br /><br />There is a bus stop right in front  , around the corner of the building that will bring you to Kowloon within mins !!!<br /><br />Super convenient area.  Quiet too !<br /><br /><b>Other things to note</b><br />there is a flatshare agreement need to sign.<br />electricity bill is not included. is shared pro-rata<br />there is refundable key deposit 500<br />stamp duty tax about 60',
 'labels': ['amenities',
  'comfortable',
  'central location',
  'peaceful',
  'strict'],
 'scores': [0.4136751890182495,
  0.274139404296875,
  0.13172298669815063,
  0.12690317630767822,
  0.05355919525027275]}